<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#pre-notebook" data-toc-modified-id="pre-notebook-0.0.1">pre notebook</a></span></li></ul></li></ul></li><li><span><a href="#prepare-OS" data-toc-modified-id="prepare-OS-1">prepare OS</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#~-snapshot-~" data-toc-modified-id="~-snapshot-~-1.0.1">~ snapshot ~</a></span></li></ul></li></ul></li><li><span><a href="#Zookeeper" data-toc-modified-id="Zookeeper-2">Zookeeper</a></span></li><li><span><a href="#IBM-Streams" data-toc-modified-id="IBM-Streams-3">IBM Streams</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#user" data-toc-modified-id="user-3.0.1">user</a></span></li><li><span><a href="#os-pre-reqs" data-toc-modified-id="os-pre-reqs-3.0.2">os pre-reqs</a></span></li><li><span><a href="#install" data-toc-modified-id="install-3.0.3">install</a></span></li><li><span><a href="#post-install" data-toc-modified-id="post-install-3.0.4">post install</a></span></li><li><span><a href="#configure" data-toc-modified-id="configure-3.0.5">configure</a></span></li><li><span><a href="#clean-up" data-toc-modified-id="clean-up-3.0.6">clean up</a></span><ul class="toc-item"><li><span><a href="#~-snapshot-~" data-toc-modified-id="~-snapshot-~-3.0.6.1">~ snapshot ~</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#ActiveMQ" data-toc-modified-id="ActiveMQ-4">ActiveMQ</a></span></li></ul></div>

### pre notebook
* fix IP 
* fix hostname
* wget https://raw.githubusercontent.com/williamClanton/gpu_neural_nets/master/neural_net_prep.ipynb
* snapshot 

# prepare OS

In [2]:
!ls  

neural_net_prep.ipynb  zookeeper-3.4.12.tar.gz	zookeeper.out


In [3]:
!uname -a

Linux rta.vion.local 3.10.0-862.6.3.el7.x86_64 #1 SMP Tue Jun 26 16:32:21 UTC 2018 x86_64 x86_64 x86_64 GNU/Linux


In [3]:
!cat /etc/redhat-release

CentOS Linux release 7.5.1804 (Core) 


In [4]:
!yum -y install epel-release

Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
 * base: mirror.wdc1.us.leaseweb.net
 * epel: ewr.edge.kernel.org
 * extras: mirror.vcu.edu
 * updates: centos2.zswap.net
Package epel-release-7-11.noarch already installed and latest version
Nothing to do


In [5]:
!yum list installed | grep -i 'java\|jdk' 

jdk1.8.x86_64                         2000:1.8.0_171-fcs               installed


In [6]:
!yum -y update

Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
 * base: mirror.wdc1.us.leaseweb.net
 * epel: fedora-epel.mirrors.tds.net
 * extras: mirror.vcu.edu
 * updates: mirror.cc.columbia.edu
No packages marked for update


### ~ snapshot ~

# Zookeeper

In [7]:
!useradd -G wheel zookeeper

In [9]:
!mkdir -p ~zookeeper/.ssh ; cp ~/.ssh/authorized_keys ~zookeeper/.ssh/ ; chown -R zookeeper:zookeeper ~zookeeper

In [10]:
!wget http://mirror.cc.columbia.edu/pub/software/apache/zookeeper/stable/zookeeper-3.4.12.tar.gz

--2018-07-13 11:59:19--  http://mirror.cc.columbia.edu/pub/software/apache/zookeeper/stable/zookeeper-3.4.12.tar.gz
Resolving mirror.cc.columbia.edu (mirror.cc.columbia.edu)... 128.59.59.71
Connecting to mirror.cc.columbia.edu (mirror.cc.columbia.edu)|128.59.59.71|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36667596 (35M) [application/x-gzip]
Saving to: ‘zookeeper-3.4.12.tar.gz’

100%[======================================>] 36,667,596  8.19MB/s   in 4.2s   

2018-07-13 11:59:24 (8.27 MB/s) - ‘zookeeper-3.4.12.tar.gz’ saved [36667596/36667596]



In [11]:
!tar xf zookeeper-3.4.12.tar.gz -C /opt

In [12]:
!mv /opt/zookeeper-3.4.12 /opt/.zookeeper-3.4.12

In [13]:
!pushd /opt; ln -s .zookeeper-3.4.12 zookeeper ; popd

/opt ~
~


In [14]:
!printf "tickTime=2000\ndataDir=/var/lib/zookeeper\nclientPort=2181\n" > /opt/zookeeper/conf/zoo.cfg 

In [15]:
!cat /opt/zookeeper/conf/zoo.cfg

tickTime=2000
dataDir=/var/lib/zookeeper
clientPort=2181


In [16]:
!mkdir -p /var/lib/zookeeper

In [17]:
!mkdir -p /var/log/zookeeper

In [18]:
!printf \
"[Unit]\n\
Description=Zookeeper\n\
After=network.target syslog.target\n\
\n\
[Service]\n\
Type=forking\n\
User=zookeeper\n\
Group=zookeeper\n\
SyslogIdentifier=zookeeper\n\
TimeoutStartSec=10min\n\
ExecStart=/opt/zookeeper/bin/zkServer.sh start\n\
ExecStop=/opt/zookeeper/bin/zkServer.sh stop\n\
SyslogIdentifier=zookeeper\n\
\n\
[Install]\n\
WantedBy=multi-user.target\n" \
> /usr/lib/systemd/system/zookeeper.service

In [19]:
!cat /usr/lib/systemd/system/zookeeper.service

[Unit]
Description=Zookeeper
After=network.target syslog.target

[Service]
Type=forking
User=zookeeper
Group=zookeeper
SyslogIdentifier=zookeeper
TimeoutStartSec=10min
ExecStart=/opt/zookeeper/bin/zkServer.sh start
ExecStop=/opt/zookeeper/bin/zkServer.sh stop
SyslogIdentifier=zookeeper

[Install]
WantedBy=multi-user.target


In [20]:
!chown -R zookeeper:zookeeper /opt/.zookeeper-3.4.12 /opt/zookeeper /var/lib/zookeeper ~zookeeper /var/log/zookeeper

In [21]:
!systemctl enable zookeeper

Created symlink from /etc/systemd/system/multi-user.target.wants/zookeeper.service to /usr/lib/systemd/system/zookeeper.service.


In [11]:
!chown -R zookeeper:zookeeper /opt/.zookeeper-3.4.12 /opt/zookeeper /var/lib/zookeeper ~zookeeper /var/log/zookeeper

In [22]:
!systemctl start zookeeper

In [23]:
!systemctl status zookeeper -l

● zookeeper.service - Zookeeper
   Loaded: loaded (/usr/lib/systemd/system/zookeeper.service; enabled; vendor preset: disabled)
   Active: inactive (dead) since Fri 2018-07-13 12:06:45 EDT; 1min 17s ago
  Process: 3794 ExecStop=/opt/zookeeper/bin/zkServer.sh stop (code=exited, status=0/SUCCESS)
  Process: 3779 ExecStart=/opt/zookeeper/bin/zkServer.sh start (code=exited, status=0/SUCCESS)

Jul 13 12:06:44 rta.vion.local systemd[1]: Starting Zookeeper...
Jul 13 12:06:44 rta.vion.local zookeeper[3779]: ZooKeeper JMX enabled by default
Jul 13 12:06:44 rta.vion.local zookeeper[3779]: Using config: /opt/zookeeper/bin/../conf/zoo.cfg
Jul 13 12:06:44 rta.vion.local zookeeper[3779]: Starting zookeeper ... /opt/zookeeper/bin/zkServer.sh: line 140: ./zookeeper.out: Permission denied
Jul 13 12:06:45 rta.vion.local zookeeper[3794]: ZooKeeper JMX enabled by default
Jul 13 12:06:45 rta.vion.local zookeeper[3794]: Using config: /opt/zookeeper/bin/../conf/zoo.cfg
Jul 13 12:06:45 rta.vion.local zookeepe

In [24]:
!systemctl start zookeeper

In [ ]:

systemctl start zookeeper
# test
systemctl status zookeeper
/opt/zookeeper/bin/zkCli.sh -server 127.0.0.1:2181
# ctrl + d to quit
rm -rf ~/zookeeper-3.4.11.tar.gz


# IBM Streams

### user

In [25]:
!useradd -G wheel streamsadmin 

In [26]:
!mkdir -p ~streamsadmin/.ssh

In [27]:
!cp /root/.ssh/authorized_keys ~streamsadmin/.ssh

In [28]:
!chown -R streamsadmin ~streamsadmin

### os pre-reqs

In [23]:
!yum -y install rdma ntp xdg-utils perl-XML-Simple gcc-c++ gcc unzip

Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
 * base: mirror.wdc1.us.leaseweb.net
 * epel: epel.mirror.constant.com
 * extras: mirror.vcu.edu
 * updates: centos2.zswap.net
Package rdma-core-15-7.el7_5.x86_64 already installed and latest version
Package ntp-4.2.6p5-28.el7.centos.x86_64 already installed and latest version
Package xdg-utils-1.1.0-0.17.20120809git.el7.noarch already installed and latest version
Package perl-XML-Simple-2.20-5.el7.noarch already installed and latest version
Package gcc-c++-4.8.5-28.el7_5.1.x86_64 already installed and latest version
Package gcc-4.8.5-28.el7_5.1.x86_64 already installed and latest version
Resolving Dependencies
--> Running transaction check
---> Package unzip.x86_64 0:6.0-19.el7 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package          Arch              Version               Repository       Size
Installing:
 unzip            x86_64            6.0-19.el7            base        

In [32]:
!systemctl enable rdma

Created symlink from /etc/systemd/system/sysinit.target.wants/rdma.service to /usr/lib/systemd/system/rdma.service.


In [33]:
!systemctl start rdma

In [36]:
!systemctl enable ntpd

Created symlink from /etc/systemd/system/multi-user.target.wants/ntpd.service to /usr/lib/systemd/system/ntpd.service.


In [37]:
!systemctl start ntpd

In [38]:
!echo "export JAVA_HOME=/usr/java/latest" >> /etc/profile.d/custom.sh
!echo "PATH=\$JAVA_HOME/bin:$PATH" >> /etc/profile.d/custom.sh
!echo "streamsadmin soft nofile 111113" >> /etc/security/limits.conf
!echo "streamsadmin hard nofile 111113" >> /etc/security/limits.conf
!echo "streamsadmin soft nproc 257262" >> /etc/security/limits.conf
!echo "streamsadmin hard nproc 257262" >> /etc/security/limits.conf
!echo "streamsadmin soft nproc 257262" >> /etc/security/limits.conf
!echo "fs.file-max = 6815744" >>  /etc/sysctl.conf
# reboot
# prolly a good spot to snapshot if virtual

In [39]:
!mkdir -p /opt/ibm/InfoSphere_Streams

In [40]:
!chown streamsadmin:streamsadmin /opt/ibm/InfoSphere_Streams

### install

In [7]:
!sudo -u streamsadmin tar xvf /home/streamsadmin/Streams-4.2.1.1-x86_64-el7.tar.gz -C /home/streamsadmin/

StreamsInstallFiles/
StreamsInstallFiles/IBMStreams_SampleResponseFile.properties
StreamsInstallFiles/com.ibm.streams.install.dependency.jar
StreamsInstallFiles/IBMStreamsSetup.bin
StreamsInstallFiles/IBMStreams-Install-Config.pdf
StreamsInstallFiles/ReleaseNotes.html
StreamsInstallFiles/dependency_checker.sh
StreamsInstallFiles/ibmdita.css


In [8]:
!~streamsadmin/StreamsInstallFiles/dependency_checker.sh


IBM Streams 4.2.1.1 Dependency Checker
Date:  July 13, 2018 3:15:57 PM EDT
Edition information:
    Edition:  IBM Streams
    Edition:  IBM Streams Developer Edition

=== System Information ===
* Hostname:  rta.vion.local
* IP address:  192.168.156.184
* Operating system:  CentOS Linux release 7.5.1804 (Core) 
* System architecture:  x86_64
* Security-Enhanced Linux setting:  Disabled
* Default Java:   
  * Java vendor:  Oracle Corporation
  * Java version:  1.8.0_171
  * Java VM version:  25.171-b11
  * Java runtime version:  1.8.0_171-b11
  * Java full version:  NOT_SET
  * Java system encoding:  NOT_SET
* Java Home:  /usr/java/latest
  * Java vendor:  Oracle Corporation
  * Java version:  1.8.0_171
  * Java VM version:  25.171-b11
  * Java runtime version:  1.8.0_171-b11
  * Java full version:  NOT_SET
  * Java system encoding:  NOT_SET
* Encoding:  UTF-8
* User Limit (ulimit -S) Settings:
  * Max processes (-u):  257262
  * Open files (-n):  111113
* User Limit (ulimit -H) Setting

In [9]:
!sudo -u streamsadmin ~streamsadmin/StreamsInstallFiles/dependency_checker.sh


IBM Streams 4.2.1.1 Dependency Checker
Date:  July 13, 2018 3:16:14 PM EDT
Edition information:
    Edition:  IBM Streams
    Edition:  IBM Streams Developer Edition

=== System Information ===
* Hostname:  rta.vion.local
* IP address:  192.168.156.184
* Operating system:  CentOS Linux release 7.5.1804 (Core) 
* System architecture:  x86_64
* Security-Enhanced Linux setting:  Disabled
* Default Java:   
  * Java vendor:  Oracle Corporation
  * Java version:  1.8.0_171
  * Java VM version:  25.171-b11
  * Java runtime version:  1.8.0_171-b11
  * Java full version:  NOT_SET
  * Java system encoding:  NOT_SET
* Java Home:  NOT_SET
* Encoding:  UTF-8
* User Limit (ulimit -S) Settings:
  * Max processes (-u):  257262
  * Open files (-n):  111113
* User Limit (ulimit -H) Settings:
  * Max processes (-u):  257262
  * Open files (-n):  111113

=== System Configuration Check ===
* Status:  PASS - Check:  IP address check
* Status:  PASS - Check:  Host name resolution check
* Status:  PASS - Ch

In [10]:
!sudo -u streamsadmin \
printf "\
RESPONSE_FILE_VERSION=2.1 \n\
INSTALLER_LOCALE=en \n\
FILE_OWNER_GROUP=streamsadmin \n\
FILE_OWNER_USER_ID=streamsadmin \n\
USER_INSTALL_DIR=/opt/ibm/InfoSphere_Streams \n\
IS_EDITION_TYPE=Product\n\
LICENSE_ACCEPTED=TRUE \n\
VERSION_EXIST_ACTION=0 \n\
MISSING_DEPENDENCY_ACTION=0 \n" \
> /home/streamsadmin/StreamsInstallFiles/installer.properties

In [11]:
!/home/streamsadmin/StreamsInstallFiles/IBMStreamsSetup.bin \
-jvmxms 32G \
-jvmxmx 32G \
-i silent \
-f /home/streamsadmin/StreamsInstallFiles/installer.properties \
-r /home/streamsadmin/StreamsInstallFiles/new.installer.properties

### post install

In [13]:
!sed -i '1 s/^/export SWT_GTK3=0\n/' /opt/ibm/InfoSphere_Streams/4.2.1.1/bin/streamsprofile.sh

In [14]:
!echo "export STREAMS_INSTALL=/opt/ibm/InfoSphere_Streams/4.2.1.1" >> /etc/profile.d/custom.sh

In [15]:
!echo "export STREAMS_SPLPATH=$STREAMS_INSTALL/toolkits/com.ibm.streamsx.messaging" >> /etc/profile.d/custom.sh

In [16]:
!cat /etc/profile.d/custom.sh

export JAVA_HOME=/usr/java/latest
PATH=$JAVA_HOME/bin:/opt/anaconda/bin:/opt/anaconda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/root/bin
export JAVA_HOME=/usr/java/latest
PATH=$JAVA_HOME/bin:/opt/anaconda/bin:/opt/anaconda/bin:/usr/java/latest/bin:/opt/anaconda/bin:/opt/anaconda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/root/bin:/root/bin
export STREAMS_INSTALL=/opt/ibm/InfoSphere_Streams/4.2.1.1
export STREAMS_SPLPATH=/toolkits/com.ibm.streamsx.messaging


In [17]:
!echo "source /opt/ibm/InfoSphere_Streams/4.2.1.1/bin/streamsprofile.sh" >> /home/streamsadmin/.bashrc

In [21]:
!echo "source /opt/ibm/InfoSphere_Streams/4.2.1.1/bin/streamsprofile.sh" >> /root/.bashrc

In [23]:
!source /root/.bashrc

### configure 

In [33]:
!sudo \
STREAMS_INSTALL=/opt/ibm/InfoSphere_Streams/4.2.1.1 \
-u streamsadmin -g streamsadmin  \
/opt/ibm/InfoSphere_Streams/4.2.1.1/bin/streamtool \
mkdomain -d StreamsDomain  --zkconnect localhost:2181

CDISA0017I Creating the following domain: StreamsDomain.
CDISA0018I The following domain was created successfully: StreamsDomain.


In [36]:
!sudo \
STREAMS_INSTALL=/opt/ibm/InfoSphere_Streams/4.2.1.1 \
-u streamsadmin -g streamsadmin  \
/opt/ibm/InfoSphere_Streams/4.2.1.1/bin/streamtool \
genkey -d StreamsDomain \
--zkconnect localhost:2181

CDISC0013I The following private key file was created for authentication: /home/streamsadmin/.streams/key/StreamsDomain/streamsadmin_priv.pem.
CDISC0014I The following public key file was created for authentication: streamsadmin.
CDISA0089I Generated key successfully for the following domain: StreamsDomain.


In [37]:
!sudo \
STREAMS_INSTALL=/opt/ibm/InfoSphere_Streams/4.2.1.1 \
-u streamsadmin -g streamsadmin  \
/opt/ibm/InfoSphere_Streams/4.2.1.1/bin/streamtool \
startdomain -d StreamsDomain  \
--zkconnect localhost:2181

CDISA0064I Starting the authentication and auditing services for the following domain: StreamsDomain.
CDISA0021I Starting the following domain: StreamsDomain.
CDISA0034I Waiting for all services to start.
CDISA0038I The aas service started on the following resource: rta.vion.local. The service has the following process id: 12655.
CDISA0038I The auditlog service started on the following resource: rta.vion.local. The service has the following process id: 12655.
CDISA0039I The jmx service started on the following resource: rta.vion.local. The service has the following process id: 13705. The service uses the following port: 9975.
CDISA0039I The sws service started on the following resource: rta.vion.local. The service has the following process id: 14164. The service uses the following port: 8443.
CDISA0022I The following domain started successfully: StreamsDomain.


In [42]:
!sudo \
STREAMS_INSTALL=/opt/ibm/InfoSphere_Streams/4.2.1.1 \
-u streamsadmin -g streamsadmin  \
/opt/ibm/InfoSphere_Streams/4.2.1.1/bin/streamtool \
mkinstance -i StreamsInstance \
-d StreamsDomain \
--zkconnect localhost:2181

CDISA0026I Creating the StreamsInstance instance for the following domain: StreamsDomain.
CDISA0027I The StreamsInstance instance was created successfully for the following domain: StreamsDomain.


In [45]:
!sudo \
STREAMS_INSTALL=/opt/ibm/InfoSphere_Streams/4.2.1.1 \
-u streamsadmin -g streamsadmin  \
/opt/ibm/InfoSphere_Streams/4.2.1.1/bin/streamtool \
startinstance \
-d StreamsDomain \
-i StreamsInstance \
--zkconnect localhost:2181

CDISA0030I Starting the StreamsInstance instance for the following domain: StreamsDomain.
CDISC0164I IBM Streams is adding the following resource specification identifier to the StreamsInstance instance: 1[1(none)[shared]]. The instance is in the StreamsDomain domain.
CDISA3019W No resources with the management tag are available in the StreamsInstance instance for the following domain: StreamsDomain. Using rta.vion.local as a management resource.
CDISA3040W No resources with the view tag are available for the StreamsInstance instance in the following domain: StreamsDomain. Using rta.vion.local as a view resource.
CDISA3013W No resources with the application tag are available in the StreamsInstance instance for the following domain: StreamsDomain. Using rta.vion.local as an application resource.
CDISA0034I Waiting for all services to start.
CDISA0038I The app service started on the following resource: rta.vion.local. The service has the following process id: 15337.
CDISA0038I The sam se

In [48]:
!sudo \
STREAMS_INSTALL=/opt/ibm/InfoSphere_Streams/4.2.1.1 \
-u streamsadmin -g streamsadmin  \
/opt/ibm/InfoSphere_Streams/4.2.1.1/bin/streamtool \
geturl \
-d StreamsDomain \
--zkconnect localhost:2181

https://rta.vion.local:8443/streams/domain/console


In [52]:
!sudo \
STREAMS_INSTALL=/opt/ibm/InfoSphere_Streams/4.2.1.1 \
-u streamsadmin -g streamsadmin  \
/opt/ibm/InfoSphere_Streams/4.2.1.1/bin/streamtool \
stopinstance \
-i StreamsInstance \
-d StreamsDomain \
--zkconnect localhost:2181

CDISA0032I Stopping the StreamsInstance instance for the following domain: StreamsDomain.
CDISA0035I Waiting for all services to stop.
CDISA0042I The view service stopped on the following resource: rta.vion.local. The service had the following process id: 15337.
CDISA0042I The app service stopped on the following resource: rta.vion.local. The service had the following process id: 15337.
CDISA0042I The srm service stopped on the following resource: rta.vion.local. The service had the following process id: 15337.
CDISA0042I The sam service stopped on the following resource: rta.vion.local. The service had the following process id: 15337.
CDISA0033I The StreamsInstance instance stopped successfully for the following domain: StreamsDomain.


In [53]:
!sudo \
STREAMS_INSTALL=/opt/ibm/InfoSphere_Streams/4.2.1.1 \
-u streamsadmin -g streamsadmin  \
/opt/ibm/InfoSphere_Streams/4.2.1.1/bin/streamtool \
stopdomain \
-d StreamsDomain \
--zkconnect localhost:2181

CDISA0023I Stopping the following domain: StreamsDomain.
CDISA0035I Waiting for all services to stop.
CDISA0042I The sws service stopped on the following resource: rta.vion.local. The service had the following process id: 14164.
CDISA0042I The jmx service stopped on the following resource: rta.vion.local. The service had the following process id: 13705.
CDISA0042I The auditlog service stopped on the following resource: rta.vion.local. The service had the following process id: 12655.
CDISA0042I The aas service stopped on the following resource: rta.vion.local. The service had the following process id: 12655.
CDISA0042I The controller service stopped on the following resource: rta.vion.local. The service had the following process id: 12455.
CDISA0024I The following domain stopped successfully: StreamsDomain.


### clean up

In [54]:
!rm -rf /home/streamsadmin/Streams-4.2.1.1-x86_64-el7.tar.gz

In [55]:
!rm -rf /home/streamsadmin/StreamsInstallFiles/

#### ~ snapshot ~

# ActiveMQ